<a href="https://colab.research.google.com/github/Kaviyarasi-Sasiperumal/AI_-Based_-Document-_Search_-and-_Knowledge-_Retrieval_-with-_Conversational_Interface/blob/main/Milestone2_RAG_Pipeline_Development.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U langchain langchain-community pypdf faiss-cpu sentence-transformers

In [ ]:
import os
os.listdir()

In [ ]:
import os

for root, dirs, files in os.walk("/content"):
    if "sample.pdf" in files:
        print("FOUND AT:", os.path.join(root,"sample.pdf"))


In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/sample_milestone_rag.pdf")
documents = loader.load()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -U langchain-text-splitters


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

chunks = text_splitter.split_documents(documents)
print("Total chunks:", len(chunks))


In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)


In [ ]:
from langchain_community.vectorstores import FAISS

vector_db = FAISS.from_documents(chunks, embeddings)
vector_db.save_local("faiss_index")

print(" Milestone 1 COMPLETED!")


In [ ]:
!pip install -q transformers torch gradio


In [ ]:
!pip uninstall -y langchain langchain-community
!pip install langchain==0.1.20 langchain-community==0.0.38 pypdf


In [ ]:
import langchain
print(langchain.__version__)

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings


In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

print("LangChain Community imported successfully ")


In [ ]:
from langchain_community.vectorstores import FAISS
print("FAISS import OK")


In [ ]:
!pip install pypdf



In [ ]:
import pypdf
print("pypdf installed successfully ")


In [ ]:
from google.colab import files
files.upload()



In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/sample_milestone_rag.pdf")
documents = loader.load()

print("Documents loaded:", len(documents))


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

chunks = text_splitter.split_documents(documents)
print("Total chunks:", len(chunks))


In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2")


In [ ]:
from langchain_community.vectorstores import FAISS

vector_db = FAISS.from_documents(chunks, embeddings)
vector_db.save_local("faiss_index")

print("Milestone 1 COMPLETED!")


In [ ]:
from langchain_community.vectorstores import FAISS

vector_db = FAISS.load_local(
    "faiss_index",
    embeddings,
    allow_dangerous_deserialization=True
)

retriever = vector_db.as_retriever(search_kwargs={"k": 3})

print("FAISS vectors:", vector_db.index.ntotal)


In [ ]:
from transformers import pipeline
import torch

llm = pipeline(
    task="text2text-generation",
    model="google/flan-t5-base",
    device=-1,
    torch_dtype=torch.float32,
    max_new_tokens=128
)



In [ ]:
def rag_answer(question):
    try:
        if not question or question.strip() == "":
            return "Please enter a question."

        docs = vector_db.similarity_search(question, k=3)

        if not docs:
            return "No relevant information found."

        context = "\n".join(
            doc.page_content for doc in docs if doc.page_content
        )

        if not context.strip():
            return "Document text is empty."

        prompt = (
            "Answer the question using ONLY the information below.\n\n"
            f"Context:\n{context}\n\n"
            f"Question:\n{question}\n\n"
            "Answer:"
        )

        output = llm(prompt)


        if isinstance(output, list) and "generated_text" in output[0]:
            return output[0]["generated_text"].strip()

        return str(output)

    except Exception as e:
        return f"Backend Error: {e}"



In [29]:
import gradio as gr

with gr.Blocks() as demo:
    gr.Markdown(" RAG-based Chatbot (Milestone 2)")

    question = gr.Textbox(label="Ask a question from the document")
    answer = gr.Textbox(label="Answer", lines=8)

    btn = gr.Button("Ask")
    btn.click(rag_answer, inputs=[question], outputs=[answer])

demo.launch(debug=True)




Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://2de9244d830c349b67.gradio.live
